In [14]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [15]:
# construct a SparkSession, giving it a relevant appName
spark = SparkSession.builder.appName(
    "Analyzing the vocabulary of Pride and Prejudice."
).getOrCreate()

In [25]:
results = (
    spark.read.text("../data/gutenberg_books/1342-0.txt")
    .select(F.split(F.col("value"), " ").alias("line"))
    .select(F.explode(F.col("line")).alias("word"))
    .select(F.lower(F.col("word")).alias("word"))
    .select(F.regexp_extract(F.col("word"), "[a-z']*", 0).alias("word"))
    .where(F.col("word") != "")
    .groupby("word")
    .count()
)

In [26]:
results.show()

+-------------+-----+
|         word|count|
+-------------+-----+
|       online|    4|
|         some|  203|
|        still|   72|
|          few|   72|
|         hope|  122|
|        those|   60|
|     cautious|    4|
|       lady's|    8|
|    imitation|    1|
|          art|    3|
|      solaced|    1|
|       poetry|    2|
|    arguments|    5|
| premeditated|    1|
|      elevate|    1|
|       doubts|    2|
|    destitute|    1|
|    solemnity|    5|
|gratification|    1|
|    connected|   14|
+-------------+-----+
only showing top 20 rows



In [27]:
results.write.csv("../data/simple_count.csv")

In [28]:
results.coalesce(1).write.csv("../data/simple_count_single_partition.csv")